<h1 style="color: #5e9ca0;"><span style="color: #2b2301;">PA Cloud </span>and&nbsp;<span style="color: #2b2301;">IEX Cloud</span> integration for Financial Data using Python</h1>

In [ ]:
##I prefer to load all my credentials from a csv file. Your call on how to load yours.
#credentials = pd.read_csv(body)

In [ ]:
!pip install tm1py
!pip install iexfinance

In [ ]:
#bring TM1py library components to your Jupyter Notebook
from TM1py.Services import TM1Service
from TM1py.Utils import Utils
from TM1py.Objects import Cube, Dimension, Hierarchy, Element, NativeView, Subset
#import datetime library
from datetime import timedelta, date, datetime
#import IEX Finance library
import iexfinance
from iexfinance.stocks import get_historical_data
#import pandas
import pandas as pd

In [ ]:
##get an IEX Account at https://www.iexcloud.io/
#insert API token below
token = ''

In [ ]:
#this is non-interactive account for PA Cloud that is in your welcome kit
#non-interactive account username user = 'username'
user=credentials.iloc[0,0]
#non-interactive account password password = 'password'
password=credentials.iloc[0,1]
#also define your tenant pTenant = 'tenantname'
pTenant=credentials.iloc[0,2]
#define TM1 server you will be using pServer = 'servername'
pServer=credentials.iloc[0,3]
#Tenant URL
base_url = 'https://' + pTenant + '.planning-analytics.ibmcloud.com/tm1/api/' + pServer
#SSLis always true for PA Cloud
ssl=True
#IBMCloud Name Space
namespace='LDAP'

In [ ]:
# Time magic with python generator
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)


# push data to TM1
with TM1Service(base_url=base_url, 
                user=user,  
                password=password,
                namespace=namespace,
                ssl=ssl) as tm1:
    # ============================
    # create dimension IEX Date
    start_date = date(2018, 1, 1)
    end_date = date(2025, 1, 1)
    elements = [Element(str(single_date), 'Numeric') for single_date in daterange(start_date, end_date)]
    hierarchy = Hierarchy('IEX Date', 'IEX Date', elements)
    dimension = Dimension('IEX Date', [hierarchy])
    if not tm1.dimensions.exists(dimension.name):
        tm1.dimensions.create(dimension)

   
    # create dimension IEX Financial Instrument
    #Tickers to create
    #Ford, GM, Honda, Toyota
    instruments = ['F', 'GM', 'HMC','TM']
    elements = [Element(instrument, 'Numeric') for instrument in instruments]
    hierarchy = Hierarchy('IEX Financial Instrument', 'IEX Financial Instrument', elements)
    dimension = Dimension('IEX Financial Instrument', [hierarchy])
    if not tm1.dimensions.exists(dimension.name):
        tm1.dimensions.create(dimension)

    # create dimension IEX Stock Prices Measure
    measures = ('open', 'high', 'low', 'close', 'volume')
    elements = [Element(measure, 'Numeric') for measure in measures]
    hierarchy = Hierarchy('IEX Stock Prices Measure', 'IEX Stock Prices Measure', elements)
    dimension = Dimension('IEX Stock Prices Measure', [hierarchy])
    if not tm1.dimensions.exists(dimension.name):
        tm1.dimensions.create(dimension)

    # create cube IEX Stock Prices
    cube = Cube('IEX Stock Prices', ['IEX Financial Instrument', 'IEX Date', 'IEX Stock Prices Measure'])
    if not tm1.cubes.exists(cube.name):
        tm1.cubes.create(cube)

In [ ]:
#change tickers here 
pTick=instruments[0]
token = token
#query data between these dates
start = datetime(2019, 7, 1)
end = datetime(2019, 7, 24)
#uncomment to lines below to pull yesterday's data only, if you schedule to run daily
#start = datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d')
#end = datetime.strftime(datetime.now(), '%Y-%m-%d')
data= get_historical_data(pTick,start,end, token=token, output_format='pandas')
data

In [ ]:
pCube='IEX Stock Prices'
cellset = {}
for tmstp, row in data.iterrows():
    date = tmstp.date()
    for measure in measures:
        cellset[(pTick, str(date), measure)] = row[measure]

# push data to TM1
with TM1Service(base_url=base_url, 
                user=user,  
                password=password,
                namespace=namespace,
                ssl=ssl) as tm1:
    tm1.cubes.cells.write_values(pCube, cellset)

In [ ]:
#optional cleanup cubes
#warning this will delete cubes you created above

with TM1Service(base_url=base_url, 
                user=user,  
                password=password,
                namespace=namespace,
                ssl=ssl) as tm1:
    cube_names = tm1.cubes.get_all_names()
    for cube_name in cube_names:
        if "IEX" in cube_name:
            tm1.cubes.delete(cube_name)

In [ ]:
#optional cleanup dimensions
#warning this will delete dimensions you created above

with TM1Service(base_url=base_url, 
                user=user,  
                password=password,
                namespace=namespace,
                ssl=ssl) as tm1:
    dimension_names = tm1.dimensions.get_all_names()
    for dimension_name in dimension_names:
        if "IEX" in dimension_name and "}" not in dimension_name:
            tm1.dimensions.delete(dimension_name)